In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-whitegrid')

import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_colwidth', -1)

import warnings
warnings.simplefilter('ignore')

C:\Users\Marc\AppData\Local\Temp\ipykernel_3024\3792999127.py:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')
C:\Users\Marc\AppData\Local\Temp\ipykernel_3024\3792999127.py:23: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
#ss=pd.read_csv('/kaggle/input/delloite-machinehack/submission.csv')

In [5]:
# feature importance file after generating lot of features(derived features using LGBM feature importance)
fets=pd.read_csv('/kaggle/input/impftds/importantfrs.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/impftds/importantfrs.csv'

In [5]:
# Concatinating train and test data for data preprocessing and feature engineering
df =pd.concat([train, test], axis=0).reset_index(drop = True)
df.shape

(364782, 18)

In [6]:
#Categorical columns
cat_cols =df.select_dtypes('object').columns
cat_cols

Index(['LoanID', 'Education', 'EmploymentType', 'MaritalStatus', 'HasMortgage',
       'HasDependents', 'LoanPurpose', 'HasCoSigner'],
      dtype='object')

In [7]:
ID_COL, TARGET_COL = 'LoanID', 'Default'
features = [c for c in df.columns if c not in [ID_COL, TARGET_COL]]

cat_cols = ['Education', 'EmploymentType', 'MaritalStatus', 'HasMortgage',
       'HasDependents', 'LoanPurpose', 'HasCoSigner']

num_cols = [c for c in features if c not in cat_cols]

In [8]:
num_cols

['Age',
 'Income',
 'LoanAmount',
 'CreditScore',
 'MonthsEmployed',
 'NumCreditLines',
 'InterestRate',
 'LoanTerm',
 'DTIRatio']

**Bining for numerical columns**

In [9]:
#bining features using num_cols
# 10 bins
df["Age_bin_10"] = pd.cut(df["Age"], bins=10, labels=False)
df["Income_bin_5"] = pd.cut(df["Income"], bins=5, labels=False)
df["LoanAmount_bin_10"] = pd.cut(df["LoanAmount"], bins=10, labels=False)
df["CreditScore_bin_10"] = pd.cut(df["CreditScore"], bins=10, labels=False)
df["MonthsEmployed_bin_5"] = pd.cut(df["MonthsEmployed"], bins=5, labels=False)
df["NumCreditLines_bin_4"] = pd.cut(df["NumCreditLines"], bins=4, labels=False)
df["InterestRate_bin_5"] = pd.cut(df["InterestRate"], bins=5, labels=False)
df["LoanTerm_bin_5"] = pd.cut(df["LoanTerm"], bins=5, labels=False)
df["DTIRatio_bin_5"] = pd.cut(df["DTIRatio"], bins=5, labels=False)

**Generating arithematic features using numerical features**

In [10]:
#set_1 using loan amount funded and funded investor amount
df['debt']=df['DTIRatio']*df['Income']
df['loan_inc_sum']=df['LoanAmount']+df['Income']
df['loan_inc_div']=df['LoanAmount']/df['Income']
df['loan_age_div']=df['LoanAmount']/df['Age']
df['loan_debt_div']=df['LoanAmount']/df['debt']

In [11]:
df['loanT_inc_sum']=df['LoanTerm']+df['Income']
df['loanT_inc_div']=df['LoanTerm']/df['Income']
df['loanT_age_div']=df['LoanTerm']/df['Age']
df['loanT_debt_div']=df['LoanTerm']/df['debt']

In [12]:
df['loan_monthly']=df['LoanAmount']/df['LoanTerm']
df['loan_monthly_int']=df['LoanAmount']/df['LoanTerm']*df['InterestRate']
df['inc_loan_monthly_sub']=df["Income"]-df['loan_monthly']
df['inc_loan_monthly_int_sub']=df["Income"]-df['loan_monthly_int']
df['inc_loan_monthly_div']=df["Income"]/df['loan_monthly']
df['inc_loan_monthly_int_div']=df["Income"]/df['loan_monthly_int']

In [13]:
#set_1 using loan amount funded and funded investor amount
df['cs_debt_div']=df['CreditScore']/df['debt']
df['cs_inc_div']=df['CreditScore']/df['Income']
df['cs_age_div']=df['CreditScore']/df['Age']
df['cs_numCredLines_div']=df['CreditScore']/df['NumCreditLines']

In [14]:
#set_2 using interest rate debit to income and home ownership
df['inte_debit_sum']=df['InterestRate']+df['DTIRatio']
df['inte_debit_divi']=df['InterestRate']/df['DTIRatio']
df['inte_debit_mult']=df['InterestRate']*df['DTIRatio']

In [15]:
df.shape

(364782, 49)

**Incrementing numerical features using PCA analysis and clustering the data into two groups**

In [16]:
from sklearn import decomposition

Principal component analysis on numerical columns

In [17]:
%%time

pca = decomposition.IncrementalPCA(n_components=9)
emb = pca.fit_transform(df[num_cols].values)
for i in range(emb.shape[1]):
    df[f'pca_{i+1}'] = emb[:, i]
    
print(df.shape)

(364782, 58)
CPU times: total: 2.33 s
Wall time: 2.42 s


In [18]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [20]:
# Fit the KMeans model to X_scaled and create the cluster labels
kmeans = KMeans(n_clusters=2, n_init=49, random_state=0)
df["Cluster"] = kmeans.fit_predict(df[num_cols].values)

**Combination of Categorical features**

In [21]:
cat_cols

['Education',
 'EmploymentType',
 'MaritalStatus',
 'HasMortgage',
 'HasDependents',
 'LoanPurpose',
 'HasCoSigner']

In [22]:
df['Edu_Empl']=df['Education']+df['EmploymentType']
df['Edu_LoanPurp']=df['Education']+df['LoanPurpose']
df['MaritalStat_LoanPurp']=df['MaritalStatus']+df['LoanPurpose']
df['MaritalStat_Empl']=df['MaritalStatus']+df['EmploymentType']
df['MaritalStat_Dep']=df['MaritalStatus']+df['HasDependents']
df['MaritalStat_Empl_Mortg']=df['MaritalStatus']+df['EmploymentType']+df['HasMortgage']
df['Empl_CoSigner']=df['EmploymentType']+df['HasCoSigner']

In [23]:
#keeping new categorical columns in a list to use further
new_cat_cols=['Edu_Empl','Edu_LoanPurp','MaritalStat_LoanPurp','MaritalStat_Empl','MaritalStat_Dep','MaritalStat_Empl_Mortg','Empl_CoSigner']

***Loading libraries***

In [41]:
import numpy as np
import pandas as pd
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.metrics import *
#from lightgbm import LGBMClassifier
import lightgbm as lgb
from catboost import CatBoostRegressor

import gc
from tqdm import *

In [26]:
# I treated Term and accounts delinquent as categorical features instea of discrete values
#cat_ord_cols=['Term','Accounts Delinquent']

In [25]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

**Aggregation features for categorical and numerical features**

1. It will take 15 minutes to get new features

In [26]:
df.columns

Index(['LoanID', 'Age', 'Income', 'LoanAmount', 'CreditScore',
       'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm',
       'DTIRatio', 'Education', 'EmploymentType', 'MaritalStatus',
       'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner', 'Default',
       'Age_bin_10', 'Income_bin_5', 'LoanAmount_bin_10', 'CreditScore_bin_10',
       'MonthsEmployed_bin_5', 'NumCreditLines_bin_4', 'InterestRate_bin_5',
       'LoanTerm_bin_5', 'DTIRatio_bin_5', 'debt', 'loan_inc_sum',
       'loan_inc_div', 'loan_age_div', 'loan_debt_div', 'loanT_inc_sum',
       'loanT_inc_div', 'loanT_age_div', 'loanT_debt_div', 'loan_monthly',
       'loan_monthly_int', 'inc_loan_monthly_sub', 'inc_loan_monthly_int_sub',
       'inc_loan_monthly_div', 'inc_loan_monthly_int_div', 'cs_debt_div',
       'cs_inc_div', 'cs_age_div', 'cs_numCredLines_div', 'inte_debit_sum',
       'inte_debit_divi', 'inte_debit_mult', 'pca_1', 'pca_2', 'pca_3',
       'pca_4', 'pca_5', 'pca_6', 'pca_7', 'pc

In [27]:
cat_fts = ['Education', 'EmploymentType', 'MaritalStatus',
 'HasMortgage', 'HasDependents', 'LoanPurpose',
 'HasCoSigner','Edu_Empl','Edu_LoanPurp',
 'MaritalStat_LoanPurp','MaritalStat_Empl','MaritalStat_Dep',
 'MaritalStat_Empl_Mortg','Empl_CoSigner']

num_fts_2 = ['Age',
 'Income',
 'LoanAmount',
 'CreditScore',
 'MonthsEmployed',
 'NumCreditLines',
 'InterestRate',
 'LoanTerm',
 'DTIRatio']

# cat_fts =['Batch Enrolled', 'Grade', 'Sub Grade', 'Employment Duration',
#        'Verification Status', 'Payment Plan', 'Loan Title',
#        'Initial List Status', 'Application Type',
#           'Batch_empduration','Grade_subgrade',
#           'status_plan','plan_loantitle','inital_aptype','verify_initial']
# num_fts_2=['Loan Amount', 'Funded Amount', 'Open Account',
#        'Revolving Balance', 'Total Accounts', 'Collection 12 months Medical',
#        'Last week Pay', 'Total Collection Amount',
#        'Total Current Balance', 'Total Revolving Credit Limit','Funded Amount Investor', 'Interest Rate', 'Home Ownership',
#        'Debit to Income', 'Revolving Utilities', 'Total Received Interest',
#        'Total Received Late Fee', 'Recoveries', 'Collection Recovery Fee','Public Record']

for g in tqdm_notebook(cat_fts):

  num_fts = [c for c in num_fts_2 if c != g]
  grp = df.groupby(g)[num_fts].mean()
  grp.columns = [c + f'_grpd_by_{g}_mean' for c in grp.columns]
  df = pd.merge(df, grp, on = g, how = 'left')

  grp = df.groupby(g)[num_fts].std()
  grp.columns = [c + f'_grpd_by_{g}_std' for c in grp.columns]
  df = pd.merge(df, grp, on = g, how = 'left')

  grp = df.groupby(g)[num_fts].min()
  grp.columns = [c + f'_grpd_by_{g}_min' for c in grp.columns]
  df = pd.merge(df, grp, on = g, how = 'left')

  grp = df.groupby(g)[num_fts].max()
  grp.columns = [c + f'_grpd_by_{g}_max' for c in grp.columns]
  df = pd.merge(df, grp, on = g, how = 'left')

  0%|          | 0/14 [00:00<?, ?it/s]

In [28]:
df.shape

(364782, 570)

Concatinating all categorical features

In [29]:
fts=cat_cols+new_cat_cols

**Dummy encoding for categorical features**

In [30]:
df=pd.get_dummies(df,columns=fts)

*Dividing train and test data*

In [118]:
train1, test1 = df[:train.shape[0]].reset_index(drop = True), df[train.shape[0]:].reset_index(drop = True)

In [105]:
#train1 = train1.drop("Default", axis = 1)
#test1 = test1.drop("Default", axis = 1)

In [106]:
train1.head(5)

,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Age_bin_10,Income_bin_5,LoanAmount_bin_10,CreditScore_bin_10,MonthsEmployed_bin_5,NumCreditLines_bin_4,InterestRate_bin_5,LoanTerm_bin_5,DTIRatio_bin_5,debt,loan_inc_sum,loan_inc_div,loan_age_div,loan_debt_div,loanT_inc_sum,loanT_inc_div,loanT_age_div,loanT_debt_div,loan_monthly,loan_monthly_int,inc_loan_monthly_sub,inc_loan_monthly_int_sub,inc_loan_monthly_div,inc_loan_monthly_int_div,cs_debt_div,cs_inc_div,cs_age_div,cs_numCredLines_div,inte_debit_sum,inte_debit_divi,inte_debit_mult,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,Cluster,Age_grpd_by_Education_mean,Income_grpd_by_Education_mean,LoanAmount_grpd_by_Education_mean,CreditScore_grpd_by_Education_mean,MonthsEmployed_grpd_by_Education_mean,NumCreditLines_grpd_by_Education_mean,InterestRate_grpd_by_Education_mean,LoanTerm_grpd_by_Education_mean,DTIRatio_grpd_by_Education_mean,Age_grpd_by_Education_std,Income_grpd_by_Education_std,LoanAmount_grpd_by_Education_std,CreditScore_grpd_by_Education_std,MonthsEmployed_grpd_by_Education_std,NumCreditLines_grpd_by_Education_std,InterestRate_grpd_by_Education_std,LoanTerm_grpd_by_Education_std,DTIRatio_grpd_by_Education_std,Age_grpd_by_Education_min,Income_grpd_by_Education_min,LoanAmount_grpd_by_Education_min,CreditScore_grpd_by_Education_min,MonthsEmployed_grpd_by_Education_min,NumCreditLines_grpd_by_Education_min,InterestRate_grpd_by_Education_min,LoanTerm_grpd_by_Education_min,DTIRatio_grpd_by_Education_min,Age_grpd_by_Education_max,Income_grpd_by_Education_max,LoanAmount_grpd_by_Education_max,CreditScore_grpd_by_Education_max,MonthsEmployed_grpd_by_Education_max,NumCreditLines_grpd_by_Education_max,InterestRate_grpd_by_Education_max,LoanTerm_grpd_by_Education_max,DTIRatio_grpd_by_Education_max,Age_grpd_by_EmploymentType_mean,Income_grpd_by_EmploymentType_mean,LoanAmount_grpd_by_EmploymentType_mean,CreditScore_grpd_by_EmploymentType_mean,MonthsEmployed_grpd_by_EmploymentType_mean,NumCreditLines_grpd_by_EmploymentType_mean,InterestRate_grpd_by_EmploymentType_mean,LoanTerm_grpd_by_EmploymentType_mean,DTIRatio_grpd_by_EmploymentType_mean,Age_grpd_by_EmploymentType_std,Income_grpd_by_EmploymentType_std,LoanAmount_grpd_by_EmploymentType_std,CreditScore_grpd_by_EmploymentType_std,MonthsEmployed_grpd_by_EmploymentType_std,NumCreditLines_grpd_by_EmploymentType_std,InterestRate_grpd_by_EmploymentType_std,LoanTerm_grpd_by_EmploymentType_std,DTIRatio_grpd_by_EmploymentType_std,Age_grpd_by_EmploymentType_min,Income_grpd_by_EmploymentType_min,LoanAmount_grpd_by_EmploymentType_min,CreditScore_grpd_by_EmploymentType_min,MonthsEmployed_grpd_by_EmploymentType_min,NumCreditLines_grpd_by_EmploymentType_min,InterestRate_grpd_by_EmploymentType_min,LoanTerm_grpd_by_EmploymentType_min,DTIRatio_grpd_by_EmploymentType_min,Age_grpd_by_EmploymentType_max,Income_grpd_by_EmploymentType_max,LoanAmount_grpd_by_EmploymentType_max,CreditScore_grpd_by_EmploymentType_max,MonthsEmployed_grpd_by_EmploymentType_max,NumCreditLines_grpd_by_EmploymentType_max,InterestRate_grpd_by_EmploymentType_max,LoanTerm_grpd_by_EmploymentType_max,DTIRatio_grpd_by_EmploymentType_max,Age_grpd_by_MaritalStatus_mean,Income_grpd_by_MaritalStatus_mean,LoanAmount_grpd_by_MaritalStatus_mean,CreditScore_grpd_by_MaritalStatus_mean,MonthsEmployed_grpd_by_MaritalStatus_mean,NumCreditLines_grpd_by_MaritalStatus_mean,InterestRate_grpd_by_MaritalStatus_mean,LoanTerm_grpd_by_MaritalStatus_mean,DTIRatio_grpd_by_MaritalStatus_mean,Age_grpd_by_MaritalStatus_std,Income_grpd_by_MaritalStatus_std,LoanAmount_grpd_by_MaritalStatus_std,CreditScore_grpd_by_MaritalStatus_std,MonthsEmployed_grpd_by_MaritalStatus_std,NumCreditLines_grpd_by_MaritalStatus_std,InterestRate_grpd_by_MaritalStatus_std,LoanTerm_grpd_by_MaritalStatus_std,DTIRatio_grpd_by_MaritalStatus_std,Age_grpd_by_MaritalStatus_min,Income_grpd_by_MaritalStatus_min,LoanAmount_grpd_by_MaritalStatus_min,CreditScore_grpd_by_MaritalStatus_mi

In [33]:
# initially I built lgbm model on 2.1k features then used lgbm feature importance to get efficient features
#fets=pd.read_csv('/kaggle/input/impftds/importantfrs.csv')

In [51]:
#features=fets.imp_fts.values.tolist()

NameError: name 'fets' is not defined

In [32]:
features_old = features.copy()

In [114]:
# remove "LoanID" and "Default" from Feature list
features = df.drop(["LoanID", "Default"], axis = 1).columns.to_list()

In [116]:
features

['Age',
 'Income',
 'LoanAmount',
 'CreditScore',
 'MonthsEmployed',
 'NumCreditLines',
 'InterestRate',
 'LoanTerm',
 'DTIRatio',
 'Age_bin_10',
 'Income_bin_5',
 'LoanAmount_bin_10',
 'CreditScore_bin_10',
 'MonthsEmployed_bin_5',
 'NumCreditLines_bin_4',
 'InterestRate_bin_5',
 'LoanTerm_bin_5',
 'DTIRatio_bin_5',
 'debt',
 'loan_inc_sum',
 'loan_inc_div',
 'loan_age_div',
 'loan_debt_div',
 'loanT_inc_sum',
 'loanT_inc_div',
 'loanT_age_div',
 'loanT_debt_div',
 'loan_monthly',
 'loan_monthly_int',
 'inc_loan_monthly_sub',
 'inc_loan_monthly_int_sub',
 'inc_loan_monthly_div',
 'inc_loan_monthly_int_div',
 'cs_debt_div',
 'cs_inc_div',
 'cs_age_div',
 'cs_numCredLines_div',
 'inte_debit_sum',
 'inte_debit_divi',
 'inte_debit_mult',
 'pca_1',
 'pca_2',
 'pca_3',
 'pca_4',
 'pca_5',
 'pca_6',
 'pca_7',
 'pca_8',
 'pca_9',
 'Cluster',
 'Age_grpd_by_Education_mean',
 'Income_grpd_by_Education_mean',
 'LoanAmount_grpd_by_Education_mean',
 'CreditScore_grpd_by_Education_mean',
 'MonthsEmp

In [35]:
fts

['Education',
 'EmploymentType',
 'MaritalStatus',
 'HasMortgage',
 'HasDependents',
 'LoanPurpose',
 'HasCoSigner',
 'Edu_Empl',
 'Edu_LoanPurp',
 'MaritalStat_LoanPurp',
 'MaritalStat_Empl',
 'MaritalStat_Dep',
 'MaritalStat_Empl_Mortg',
 'Empl_CoSigner']

In [117]:
len(features)

677

In [119]:
y=train1['Default']

In [79]:
#train1[features].info()
#train1[features].select_dtypes('object').columns

Index([], dtype='object')

**Hyperparameter tuning using optuna**

In [38]:
import optuna

In [56]:
def objective(trial,data=train1[features],target=y):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=42)
    # param = {
    #     'device':'gpu',  # this parameter means using the GPU when training our model to speedup the training process
    #     'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
    #     'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
    #     'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
    #     'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
    #     'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02]),
    #     'n_estimators': 1000,
    #     'num_leaves': trial.suggest_int('num_leaves', 34, 300),
    #     'min_child_samples':trial.suggest_int('min_child_samples', 100, 500),
    #     'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17,20]),
    #     'random_state': trial.suggest_categorical('random_state', [24, 48,2020]),
    #     'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    # }
    
    # param = {
    #     'max_bin' : [128],
    #     'num_leaves': [8],
    #     'reg_alpha' : [1.2],
    #     'reg_lambda' : [1.2],
    #     'min_data_in_leaf' : [50],
    #     'bagging_fraction' : [0.5],
    #     'learning_rate' : [0.001]
    # }

    param = {
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 1000,
        'num_leaves': trial.suggest_int('num_leaves', 34, 300),
        'min_child_samples':trial.suggest_int('min_child_samples', 100, 500),
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17,20]),
        'random_state': trial.suggest_categorical('random_state', [24, 48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(train_x,train_y, eval_set=[(test_x,test_y)], eval_metric='auc')
    
    preds = model.predict_proba(test_x)[:,1]
    loss = log_loss(test_y, preds)
    return loss

In [58]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2023-09-01 15:17:13,423] A new study created in memory with name: no-name-ca45e804-38e3-4da7-82fc-d096f7693cc9


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 23753, number of negative: 180524
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.105969 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12327
[LightGBM] [Info] Number of data points in the train set: 204277, number of used features: 478
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.116278 -> initscore=-2.028155
[LightGBM] [Info] Start training from score -2.028155
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positi

[I 2023-09-01 15:18:08,527] Trial 0 finished with value: 0.0008298851751635841 and parameters: {'reg_lambda': 0.25180283987164165, 'reg_alpha': 0.0013397165279657398, 'colsample_bytree': 1.0, 'subsample': 1.0, 'learning_rate': 0.016, 'num_leaves': 291, 'min_child_samples': 453, 'max_depth': 11, 'random_state': 2020, 'min_child_weight': 85}. Best is trial 0 with value: 0.0008298851751635841.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 23753, number of negative: 180524
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.345781 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12327
[LightGBM] [Info] Number of data points in the train set: 204277, number of used features: 478
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.116278 -> initscore=-2.028155
[LightGBM] [Info] Start training from score -2.028155
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits 

[I 2023-09-01 15:19:03,035] Trial 1 finished with value: 0.0033774591156807304 and parameters: {'reg_lambda': 0.26303234397039016, 'reg_alpha': 0.013601978428504191, 'colsample_bytree': 0.4, 'subsample': 0.6, 'learning_rate': 0.01, 'num_leaves': 259, 'min_child_samples': 159, 'max_depth': 15, 'random_state': 24, 'min_child_weight': 206}. Best is trial 0 with value: 0.0008298851751635841.


...

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 23753, number of negative: 180524
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.394880 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12327
[LightGBM] [Info] Number of data points in the train set: 204277, number of used features: 478
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.116278 -> initscore=-2.028155
[LightGBM] [Info] Start training from score -2.028155
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits 

[I 2023-09-01 15:34:46,263] Trial 18 finished with value: 0.0012707183941626755 and parameters: {'reg_lambda': 0.001393281589350433, 'reg_alpha': 0.00358165661874424, 'colsample_bytree': 0.8, 'subsample': 0.5, 'learning_rate': 0.02, 'num_leaves': 232, 'min_child_samples': 240, 'max_depth': 5, 'random_state': 2020, 'min_child_weight': 130}. Best is trial 16 with value: 2.0972712665043625e-05.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 23753, number of negative: 180524
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.365719 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12327
[LightGBM] [Info] Number of data points in the train set: 204277, number of used features: 478
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.116278 -> initscore=-2.028155
[LightGBM] [Info] Start training from score -2.028155
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits 

[I 2023-09-01 15:35:35,640] Trial 19 finished with value: 0.0005488845417030891 and parameters: {'reg_lambda': 0.03251811672635375, 'reg_alpha': 0.007949259725398915, 'colsample_bytree': 0.7, 'subsample': 0.6, 'learning_rate': 0.02, 'num_leaves': 91, 'min_child_samples': 392, 'max_depth': 11, 'random_state': 48, 'min_child_weight': 56}. Best is trial 16 with value: 2.0972712665043625e-05.


Number of finished trials: 20
Best trial: {'reg_lambda': 0.01222674224146051, 'reg_alpha': 0.0012071057394647182, 'colsample_bytree': 0.6, 'subsample': 0.5, 'learning_rate': 0.02, 'num_leaves': 183, 'min_child_samples': 249, 'max_depth': 13, 'random_state': 24, 'min_child_weight': 2}


In [59]:
Best_trial={'reg_lambda': 0.01222674224146051, 'reg_alpha': 0.0012071057394647182, 'colsample_bytree': 0.6, 'subsample': 0.5, 'learning_rate': 0.02, 'num_leaves': 183, 'min_child_samples': 249, 'max_depth': 13, 'random_state': 24, 'min_child_weight': 2}

In [60]:
fit_params = {'verbose': 0, 'early_stopping_rounds': 100}

In [120]:
def run_clf_kfold(clf,fit_params, train, test, features):

  N_SPLITS = 5

  oofs = np.zeros(len(train))
  preds = np.zeros((len(test)))

  folds = StratifiedKFold(n_splits = N_SPLITS)

  for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train[TARGET_COL])):
    print(f'\n------------- Fold {fold_ + 1} -------------')

    ############# Get train, validation and test sets along with targets ################
  
    ### Training Set
    X_trn, y_trn = train[features].iloc[trn_idx], y.iloc[trn_idx]

    ### Validation Set
    X_val, y_val = train[features].iloc[val_idx], y.iloc[val_idx]

    ### Test Set
    X_test = test[features]

    ############# Scaling Data ################
    scaler = StandardScaler()
    _ = scaler.fit(X_trn)

    X_trn = scaler.transform(X_trn)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)


    ############# Fitting and Predicting ################

    _ = clf.fit(X_trn, y_trn)

    ### Instead of directly predicting the classes we will obtain the probability of positive class.
    preds_val = clf.predict_proba(X_val)[:, 1]
    preds_test = clf.predict_proba(X_test)[:, 1]

    fold_score = log_loss(y_val, preds_val)
    print(f'\nloss score for validation set is {fold_score}')

    oofs[val_idx] = preds_val
    preds += preds_test / N_SPLITS


  oofs_score = log_loss(y, oofs)
  print(f'\n\nloss score for oofs is {oofs_score}')

  return oofs, preds

In [62]:
lgbm=LGBMClassifier(**Best_trial)

In [63]:
lgb_oofs, lgb_preds =run_clf_kfold(lgbm,fit_params, train1, test1, features)


------------- Fold 1 -------------
[LightGBM] [Info] Number of positive: 23722, number of negative: 180555
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.397493 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12462
[LightGBM] [Info] Number of data points in the train set: 204277, number of used features: 478
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.116127 -> initscore=-2.029633
[LightGBM] [Info] Start training from score -2.029633
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [W

**Predictions using Gradient boosting and it will take around 40 minutes to get predictions**

In [66]:
prediction_df = pd.DataFrame({'LoanID': test[['LoanID']].values[:, 0],
                             'predicted_probability': lgb_preds})

print(prediction_df.shape)
prediction_df.head(10)

(109435, 2)


,LoanID,predicted_probability
0,7RYZGMKJIR,0.026762
1,JDL5RH07AM,0.021154
2,STAL716Y79,0.021476
3,SO0KKJ3IQB,0.033752
4,T99CWTYDCP,0.027888
5,0SNHFWV4UP,0.026885
6,S6ITP6LGYS,0.021946
7,A6I7U12IRJ,0.026269
8,8W6KY50JU4,0.035768
9,THFQ08OLMU,0.031001


In [72]:
prediction_df.to_csv("prediction_lgbm.csv", index=False)

In [68]:
from sklearn.ensemble import GradientBoostingClassifier
grade=GradientBoostingClassifier()

In [121]:
def run_clf_kfold(clf, train, test, features):

  N_SPLITS = 5

  oofs = np.zeros(len(train))
  preds = np.zeros((len(test)))

  folds = StratifiedKFold(n_splits = N_SPLITS)

  for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train[TARGET_COL])):
    print(f'\n------------- Fold {fold_ + 1} -------------')

    ############# Get train, validation and test sets along with targets ################
  
    ### Training Set
    X_trn, y_trn = train[features].iloc[trn_idx], y.iloc[trn_idx]

    ### Validation Set
    X_val, y_val = train[features].iloc[val_idx], y.iloc[val_idx]

    ### Test Set
    X_test = test[features]

    ############# Scaling Data ################
    scaler = StandardScaler()
    _ = scaler.fit(X_trn)

    X_trn = scaler.transform(X_trn)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)


    ############# Fitting and Predicting ################

    _ = clf.fit(X_trn, y_trn)

    ### Instead of directly predicting the classes we will obtain the probability of positive class.
    preds_val = clf.predict_proba(X_val)[:, 1]
    preds_test = clf.predict_proba(X_test)[:, 1]

    fold_score = log_loss(y_val, preds_val)
    print(f'\nloss score for validation set is {fold_score}')

    oofs[val_idx] = preds_val
    preds += preds_test / N_SPLITS


  oofs_score = log_loss(y, oofs)
  print(f'\n\nloss score for oofs is {oofs_score}')

  return oofs, preds

In [122]:
grade_oofs, grade_preds =run_clf_kfold(grade, train1, test1, features)


------------- Fold 1 -------------

loss score for validation set is 0.3129801409890844

------------- Fold 2 -------------

loss score for validation set is 0.31296764644689584

------------- Fold 3 -------------


In [ ]:
#ss['Loan Status']=grade_preds
#ss.to_csv('/kaggle/working/gradefinl.csv',index=False)

**Predictions using CatBoostClassifier**

In [70]:
from catboost import CatBoostClassifier
#cat=CatBoostClassifier(task_type="GPU")
cat=CatBoostClassifier()
fit_params = {'verbose': 0, 'early_stopping_rounds': 100}

In [71]:
def run_clf_kfold(clf,fit_params, train, test, features):

  N_SPLITS = 5

  oofs = np.zeros(len(train))
  preds = np.zeros((len(test)))

  folds = StratifiedKFold(n_splits = N_SPLITS)

  for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train[TARGET_COL])):
    print(f'\n------------- Fold {fold_ + 1} -------------')

    ############# Get train, validation and test sets along with targets ################
  
    ### Training Set
    X_trn, y_trn = train[features].iloc[trn_idx], y.iloc[trn_idx]

    ### Validation Set
    X_val, y_val = train[features].iloc[val_idx], y.iloc[val_idx]

    ### Test Set
    X_test = test[features]

    ############# Scaling Data ################
    scaler = StandardScaler()
    _ = scaler.fit(X_trn)

    X_trn = scaler.transform(X_trn)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)


    ############# Fitting and Predicting ################

    _ = clf.fit(X_trn, y_trn)

    ### Instead of directly predicting the classes we will obtain the probability of positive class.
    preds_val = clf.predict_proba(X_val)[:, 1]
    preds_test = clf.predict_proba(X_test)[:, 1]

    fold_score = log_loss(y_val, preds_val)
    print(f'\nloss score for validation set is {fold_score}')

    oofs[val_idx] = preds_val
    preds += preds_test / N_SPLITS


  oofs_score = log_loss(y, oofs)
  print(f'\n\nloss score for oofs is {oofs_score}')

  return oofs, preds

In [ ]:
cat_oofs, cat_preds = run_clf_kfold(cat, fit_params, train1, test1, features)

In [58]:
ss['Loan Status']=cat_preds
ss.to_csv('/kaggle/working/catfinl.csv',index=False)